## APPLE STOCK ANALYSIS AND PREDICTION 

In [1]:
#Libraries
import tensorflow
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
import yfinance as yf

# For time stamps
from datetime import datetime

In [2]:
#Bokeh Libraries
from bokeh.plotting import figure
from bokeh.io import output_file, show
from bokeh.sampledata.iris import flowers
from bokeh.models import Range1d, PanTool, ResetTool, HoverTool, ColumnDataSource, LabelSet
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models import DatetimeTickFormatter
from bokeh.models import Label

from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure 
output_notebook()

Loading BokehJS ...

Downloading Daily data from Yahoo Finance from 2000-2020. Apple went public on December 12, 1980 at 22 USD price. The stock has split 4 times, adjusting the price with split-share price is 0.39 USD We are using only past 20 years data, focus is more on last decade post 2008 recession. Apple became a 2 Trillion Dollar company in Sepetember 2020, making it the Most Valuable publicly traded company. It trades with TICKER SYMBOL AAPL.

In [4]:
df_app=yf.download('AAPL',start="2000-12-01", end="2021-06-07",interval = "1d")

[*********************100%***********************]  1 of 1 completed


Yahoo Finance gives daily data with Open, High, Low, Close and Adjusted Closing price. We collect from December 1st, 2000 to 4th November, 2020(giving future date doesnt give an error, but easier to collect latest data).

In [5]:
df_app.info

<bound method DataFrame.info of                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2000-12-01    0.303571    0.312500    0.300223    0.304688    0.262002   
2000-12-04    0.306920    0.306920    0.293527    0.297991    0.256243   
2000-12-05    0.302455    0.311384    0.292411    0.303571    0.261042   
2000-12-06    0.261161    0.267857    0.250000    0.255580    0.219774   
2000-12-07    0.257813    0.265625    0.250000    0.255580    0.219774   
...                ...         ...         ...         ...         ...   
2021-05-28  125.570000  125.800003  124.550003  124.610001  124.610001   
2021-06-01  125.080002  125.349998  123.940002  124.279999  124.279999   
2021-06-02  124.279999  125.239998  124.050003  125.059998  125.059998   
2021-06-03  124.680000  124.849998  123.129997  123.540001  123.540001   
2021-06-04  124.070000  126.160004  123.849998  125.889999  125.889999   

     

In [6]:
df_app.shape

(5158, 6)

In [7]:
df_app.describe()

,Open,High,Low,Close,Adj Close,Volume
count,5158.000000,5158.000000,5158.000000,5158.000000,5158.000000,5.158000e+03
mean,21.787272,22.018055,21.548140,21.790547,20.699111,4.366301e+08
std,28.408069,28.735195,28.055406,28.404026,28.270734,3.816266e+08
min,0.231964,0.235536,0.227143,0.234286,0.201463,3.934000e+07
25%,2.215179,2.254911,2.188839,2.213929,1.903764,1.641288e+08
50%,12.023214,12.149464,11.931250,12.051072,10.362749,3.219314e+08
75%,29.126874,29.358750,28.886875,29.076875,26.996854,5.890409e+08
max,143.600006,145.089996,141.369995,143.160004,142.704010,3.372970e+09


In [8]:
#df_app.head()
df_app=df_app.reset_index()
df_app.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-12-01,0.303571,0.312500,0.300223,0.304688,0.262002,385705600
1,2000-12-04,0.306920,0.306920,0.293527,0.297991,0.256243,371520800
2,2000-12-05,0.302455,0.311384,0.292411,0.303571,0.261042,613978400
3,2000-12-06,0.261161,0.267857,0.250000,0.255580,0.219774,1374464000
4,2000-12-07,0.257813,0.265625,0.250000,0.255580,0.219774,408917600


In [9]:
df_app['Date']=pd.to_datetime(df_app['Date'],format="%m/%d/%Y") #Changing data type and format of the date col

Graph Function will be called several times with filtered dataframe. We will be plotting Date Vs Closing Price. This function has bokeh properties defined. Since this is Time Series data, xaxis is always Year, yaxis is Price. The text color, font, font size and alignment can be changed. Refer to Bokeh Documentation.

To keep track of the highest and the lowest stock price for any given period of time, it is highlighted with green and red dots respectively. The Hover tools gives the flexibility to hover through the line and trace the changinf price.

In [10]:
def graph(df,p):
    #print (df.head())
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = 'Price'

    p.title.text_color = "black"
    p.title.text_font = "calibri"
    p.title.text_font_size = "20px"
    p.title.align = "center"

    p.xaxis.minor_tick_line_color = "black"
    p.yaxis.major_label_orientation = "vertical"
    p.xaxis.visible = True
    p.xaxis.minor_tick_in = -6
    
    
    #x axis is year, y is the price
    y2=(df['Close'].max())
    ind=df['Close'].idxmax()
    x2=df['Date'][ind].date()
    name="$"+str(round(y2,2))+", "+str(x2)
    p.circle(x2,df['Close'].max(),size = 10, color = 'green',legend_label=name)

    p.legend.location = "top_left"
    y1=(df['Close'].min())
    ind=df['Close'].idxmin()
    x1=df['Date'][ind].date()
    name="$"+str(round(y1,2))+", "+str(x1)
    p.circle(x1,y1,size = 10, color = 'red',legend_label=name)
      
    x=df.Date
    y=df.Close
    p.line(x,y,line_color="purple", line_width = 3)
    
    p.xaxis.formatter=DatetimeTickFormatter(
        days= ["%b %y"],
        months=["%b %y"],
        years=["%b %y"]
    )

    p.add_tools(HoverTool(tooltips=[
        #('date','@x'),
        ('close',  '$@{y}')], # use @{ } for field names with spaces   
                          formatters={
                         #    'date':'datetime',
                              'adj close' : 'printf', 
                          },  # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    
                          mode='vline' # display a tooltip whenever the cursor is vertically in line with a glyph
                         ))

Below are attributes that specific to each graph, such as title, dataframe and name of the html file. Hence they cannot be included in the graph function.

In [11]:
TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom,tap"
#output_file("line_chart.html", title="Line Chart")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="",title="APPLE")
#p.title.text = "APPLE"
graph(df_app,p)
show( p , notebook_handle=True) 

###### 1. Apple time series from 2000-2020 plot.

Apple stock has grown steadily since 1980. For this dataset, the minimum price was on 17th April, 2003 at 0.23 USD. The highest price was on 1st September 2020. On August 19th 2020, Apple became a 2 Trillion Dollar company. It took 42 years to reach 1 Trillion market cap on 2nd August 2018 and just over 24 months to reach 2 Trillion Dollar Market cap. iPhone got Apple to 1 Trillion and Services got Apple to 2 Trillion. In 2020, midst of pandemic, apple shares have increased by 50% in price. 

https://www.theverge.com/2020/8/19/21375223/apple-2-trillion-market-cap-first-us-company-stock-price

In [12]:
df_app['Change']=df_app['High']-df_app['Low']
df_app['C']=df_app['Open']-df_app['Close']

In [15]:
df_1120=df_app[(df_app['Date']>"2010-12-31")]
output_file("APPLE_2011_2020.html", title="APPLE 2011-2012")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2011-2020"
graph(df_1120,p)
show( p , notebook_handle=True) 

Apple makes announcement of new devices in September and the product release in October.
iphone 5	21-09-2012,
iphone 6	2014-09-09,
iphone 6s	25-09-2015,
iphone 7	16-09-2016,
iphone 8	22-09-2017,
iphone xr	26-10-2018,
iphone 11	20-09-2019

Its worth noting if the stock price varies with the announcement or release of new products/updates. Recently political, signing new deal and other factors have more impact that Apples product announcements. 

### Apple Yearly Analysis

In [14]:
df_2011=df_app[(df_app['Date']>"2010-12-31") & (df_app['Date']<"2012-01-01")]
output_file("APPLE_2011.html", title="APPLE 2011")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2011"
graph(df_2011,p)
show( p , notebook_handle=True) 

On 5th October 2011, Steve Jobs passed away making Timm Cook the CEO. The stock price has increased since early 2000s. In October 2011, iPhone 4s was released. October 14th the product was released, 4 million units were sold, double the sales of iphone 4. This led to increase in share price towards end of October.

https://techcrunch.com/2011/12/30/apple-2011/

In [16]:
sub_2012=df_app[(df_app['Date']>"2011-12-31") & (df_app['Date']<"2013-01-01")]
output_file("APPLE_2012.html", title="APPLE 2012")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2012"
graph(sub_2012,p)
show( p , notebook_handle=True) 

March 5, 2012—Apple  announced that more than 25 billion apps have been downloaded from its revolutionary App Store by the users of the more than 315 million iPhone, iPad and iPod touch devices worldwide. 


https://www.apple.com/in/newsroom/2012/03/05Apples-App-Store-Downloads-Top-25-Billion/

On March 7th Apple Brings 1080p High Definition to New Apple TV, launched new iPad.Since its launch, iPad announced it had sold 3 million units. In June it announced MacBook Pro with Retina display.
Apple Release IOS 6 on 19th September 2012. The stock was highest for that year at $25.08 on that day. It also announced iPhone 5, iPod touch and iPod Nano in September. iPhone 5 sales in the first weekend was 5 million units. Apple products sold 2 millions units of iPhone 5 in China in December.

In [17]:
sub=df_app[(df_app['Date']>"2012-12-31") & (df_app['Date']<"2014-01-01")]
output_file("APPLE_2013.html", title="APPLE 2013")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2013"
graph(sub,p)
show( p , notebook_handle=True) 

Apple showed slow in sales growth for 3 quarters. The trend was slowly moving up. In July, Paul Deneve was appointed as new Vice President.
But September showed a slow acceleration. September saw lot of new product releases. 10th September iPhone 5C and iPhone 5s. The first weekend of its release, Apple sold 9 million units. 18th September, iOS 7 was released. Post 19th September, the price steadily increased. In October, Angela Ahrendt was appointed as new CEO. Ipad Air and iPad Mini were released. Apple sold 33.8 million units of iPhones and 14.1 million units of iPad.

Apple signed a deal with China Mobile that resulted in 31% increase in Apple share price. It had access to China Mobile's 700 million subscribers with the new deal.

- https://www.bbc.com/news/business-25489430
- https://www.cultofmac.com/257364/entire-year-apple-news-one-post/
- https://www.theverge.com/2013/12/22/4710262/apple-iphone-china-mobile-deal


In [19]:
sub=df_app[(df_app['Date']>"2013-12-31") & (df_app['Date']<"2015-01-01")]
#output_file("APPLE_2014.html", title="APPLE 2014")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2014"
graph(sub,p)
show( p , notebook_handle=True) 

Apple purchases have been high during Thanksigiving weekend. In 2014, the high was on 26th November. It was predicted to reach its highest by many analysts.
On November 25th Apple was said to ship nearly 72 million phones for the holiday quarter. In 2014, apple had release its new iphone model, iphone 6. Iphone 5 was released 2 year prior and hence there was expectedly more rush for new model.
- https://invezz.com/news/2014/11/25/apple-share-price-company-poised-to-sell-71-5-million-iphones-in-q1/
- https://fortune.com/2014/11/11/apple-the-thanksgiving-effect/

In [20]:
sub=df_app[(df_app['Date']>"2014-12-31") & (df_app['Date']<"2016-01-01")]
output_file("APPLE_2015.html", title="APPLE 2015")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2015"
graph(sub,p)
show( p , notebook_handle=True) 

Apple hit peak in February but it continued it streak in April till early August. The iphone 6 series generated nearly 2/3rd of the revenue.
- https://www.fool.com/investing/general/2015/08/11/3-reasons-apple-stock-could-rise.aspx
- https://www.valuewalk.com/2015/04/apple-stock-on-earnings-release/

It announced a €1.7 billion plan to build and operate two data centres in Europe, each powered by 100 percent renewable energy. The facilities, located in County Galway, Ireland, and Denmark’s central Jutland, will power Apple’s online services including the iTunes Store, the App Store, iMessage, Maps and Siri for customers across Europe.
- https://www.apple.com/in/newsroom/2015/02/23Apple-to-Invest-1-7-Billion-in-New-European-Data-Centres/

In [21]:
sub=df_app[(df_app['Date']>"2015-12-31") & (df_app['Date']<"2017-01-01")]
output_file("APPLE_2016.html", title="APPLE 2016")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2016"
graph(sub,p)
show( p , notebook_handle=True) 

25 October 2016 — Apple® today announced financial results for its fiscal 2016 fourth quarter ended 24 September 2016. The Company posted quarterly revenue of 46.9 billion and quarterly net income of 9 billion, or 1.67 per diluted share. These results compare to revenue of 51.5 billion and net income of 11.1 billion, or 1.96 per diluted share, in the year-ago quarter. Gross margin was 38 percent compared to 39.9 percent in the year-ago quarter. International sales accounted for 62 percent of the quarter’s revenue.
- https://www.apple.com/in/newsroom/2016/10/apple-reports-fourth-quarter-results/

In [22]:
sub=df_app[(df_app['Date']>"2016-12-31") & (df_app['Date']<"2018-01-01")]
output_file("APPLE_2017.html", title="APPLE 2017")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2017"
graph(sub,p)
show( p , notebook_handle=True) 

Apple soared through 2017. Its lowest stock price was on the 4th Jan, the first day of the year. Since then it only went up through the year. Apple also release lot of new devices - iPad 5th Gen in March, iPad Pro, new model of MacBookPro, Macbook Air was release in June, iMac in June, Apple TV in December, iPhone 8 and iPhone 8 Plus and Apple Watch Series 3. The highest stock value was in December 18th. 
- https://www.fool.com/investing/2018/01/24/why-apple-inc-stock-soared-46-in-2017.aspx
- https://www.macrumors.com/2017/12/28/everything-apple-introduced-in-2017/

In [23]:
sub=df_app[(df_app['Date']>"2017-12-31") & (df_app['Date']<"2019-01-01")]
output_file("APPLE_2018.html", title="APPLE 2018")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2018"
graph(sub,p)
show( p , notebook_handle=True) 

Apple hit 1 Trillion Dollar on August 2nd. 

- https://markets.businessinsider.com/news/stocks/apple-stock-price-trillion-dollar-market-cap-us-company-2018-8-1027427055?_ga=2.33264958.1693862620.1600683040-887315675.1598188175#

iPhone XR was released on September 22nd. Apple shares are higher in the US Wednesday, as it has agreed a deal with one if its major chip makers, Dialog Semiconductor, to buy patents and staff in the largest such deal for the iPhone maker. The deal also includes long-term supplier contracts for Dialog Semiconductor with the US tech giant.
July quarter was the best quarter for Apple. 

But since October, the stock price fell for rest of the year. Apple shares closed higher in the US Thursday, but are lower pre-market as its fourth quarter earnings showed it sold fewer iPhones in the July-September period. The tech innovator also reported forecast beating earnings and revenues, amid a higher than expected average iPhone sale price.

- https://www.bbc.com/news/business-46281768
- https://invezz.com/news/2018/10/11/apple-shares-rise-as-it-agrees-deal-for-iphone-tech/
- https://invezz.com/news/2018/11/02/apple-shares-seen-lower-after-results-show-iphone-sales-miss/

In [24]:
sub=df_app[(df_app['Date']>"2018-12-31") & (df_app['Date']<"2020-01-01")]
output_file("APPLE_2019.html", title="APPLE 2019")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2019"
graph(sub,p)
show( p , notebook_handle=True) 

Apple did relatively better in 2019. Its lowest stock price was at the beginning of the year and it has surged since then with the highest price on 31st December. In January there was loss in sales of iPhone due to the ongoing US-China trade war. 

The year 2019 was a busy one for Apple, spreading countless hardware and software releases out through the entire year. Ranging from several updates to the Mac lineup and the iPhone 11 Pro to the Apple Watch Series 5 and AirPods Pro, Apple had updates and releases across all of its product lineups in 2019.

- https://www.macworld.com/article/3489506/apple-2019-year-in-review.html
- https://9to5mac.com/2019/12/28/apple-announcements-2019/#:~:text=The%20year%202019%20was%20a,its%20product%20lineups%20in%202019.

In [23]:
sub=df_app[(df_app['Date']>"2019-12-31") ]
output_file("APPLE_2020.html", title="APPLE 2020")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2020"
graph(sub,p)
show( p , notebook_handle=True) 

Apple become the first US company to hit a market cap of 2 trillion dollar. It’s an arbitrary milestone but a significant one all the same, testimony to the pandemic-defying performance of the iPhone maker. It’s also been just two years since Apple hit a $1 trillion market cap, meaning the company has essentially doubled in value in just over 24 months. 

https://www.businessinsider.in/stock-market/news/apple-hits-2-trillion-market-cap-becomes-first-us-listed-company-to-reach-milestone/articleshow/77638152.cms#:~:text=Apple%20just%20became%20the%20first,a%20%242%20trillion%20market%20value.

https://www.theverge.com/2020/8/19/21375223/apple-2-trillion-market-cap-first-us-company-stock-price

In [25]:
date=["19-09-2012","26-11-2014","23-02-2015","25-10-2016","18-12-2017","2018-03-10","31-12-2019","2020-09-01"]
price=[25.8,29.75,33.25,29.56,44,58.02,73.4,134.18]

d_max={'Date':date,'Price':price}
df_max=pd.DataFrame(d_max)

df_max['Date']=pd.to_datetime(df_max['Date'])
df_max

,Date,Price
0,2012-09-19,25.80
1,2014-11-26,29.75
2,2015-02-23,33.25
3,2016-10-25,29.56
4,2017-12-18,44.00
5,2018-03-10,58.02
6,2019-12-31,73.40
7,2020-09-01,134.18


In [26]:
def max_min(df,p):
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = 'Price'

    p.title.text_color = "black"
    p.title.text_font = "calibri"
    p.title.text_font_size = "20px"
    p.title.align = "center"

    p.xaxis.minor_tick_line_color = "black"
    p.yaxis.major_label_orientation = "vertical"
    p.xaxis.visible = True
    p.xaxis.minor_tick_in = -6
    
    
    #x axis is year, y is the price
    y2=(df['Price'].max())
    ind=df['Price'].idxmax()
    x2=df['Date'][ind].date()
    name="$"+str(round(y2,2))+", "+str(x2)
    p.circle(x2,y2,size = 10, color = 'green',legend_label=name)

    p.legend.location = "top_left"
    y1=(df['Price'].min())
    ind=df['Price'].idxmin()
    x1=df['Date'][ind].date()
    name="$"+str(round(y1,2))+", "+str(x1)
    p.circle(x1,y1,size = 10, color = 'red',legend_label=name)
      
    x=df.Date
    y=df.Price
    p.line(x,y,line_color="purple", line_width = 3)
    
    p.xaxis.formatter=DatetimeTickFormatter(
        days= ["%b %y"],
        months=["%b %y"],
        years=["%b %y"]
    )

    p.add_tools(HoverTool(tooltips=[
        #('date','@{x %m-%y}'),
        ('close',  '$@{y}' )], # use @{ } for field names with spaces   
                          formatters={
         #                     'date':'datetime',
                              'adj close' : 'printf', 
                          },  # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    
                          mode='vline' # display a tooltip whenever the cursor is vertically in line with a glyph
                         ))   

In [27]:
output_file("APPLE_MAX_MIN.html")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE MAX GRAPH"
max_min(df_max,p)
show( p , notebook_handle=True) 

In [28]:
df_1720=df_app[df_app['Date']>'2016-12-31']
sub=df_1720
#output_file("APPLE_2016_2020.html", title="APPLE 2017-2020")
p = figure(x_axis_type='datetime', y_axis_type="linear",plot_height = 500,plot_width = 1000,tools="")
p.title.text = "APPLE 2017-2020"
graph(sub,p)
show( p , notebook_handle=True) 